This document is a Python exploration of this R-based document: http://m-clark.github.io/data-processing-and-visualization/.  It is intended for those new to modeling and related concepts.  Code is *not* optimized for anything but learning.  In addition, all the content is located with the main document, not here, so many sections may not be included.  I only focus on reproducing the code chunks and providing some useful context.

In [1]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
import pandas as pd
import numpy as np

In [6]:
from sklearn.model_selection import train_test_split
# from sklearn import datasets
from sklearn.ensemble import RandomForestRegressor
# from sklearn.ensemble import GradientBoostingRegressor

In [7]:
happy = pd.read_csv('../data/world_hapiness.csv')

def scale(x):
    return((x - np.nanmean(x, axis=0))/np.nanstd(x, axis=0))

happy_processed = happy[[
    'country', 
    'year', 
    'happiness_score',
    'democratic_quality',
    'generosity',
    'healthy_life_expectancy_at_birth',
    'log_gdp_per_capita'
]].apply(
    lambda x: scale(x)
    if x.name in [
        'happiness_score',
        'democratic_quality',
        'generosity',
        'healthy_life_expectancy_at_birth',] 
    else x
).assign(
    year = happy.year - np.min(happy.year)
).dropna()

happy_processed.head()

,country,year,happiness_score,democratic_quality,generosity,healthy_life_expectancy_at_birth,log_gdp_per_capita
7,Afghanistan,10,-1.625550,-1.950587,0.545034,-1.307413,7.500539
8,Afghanistan,11,-1.815967,-1.963219,0.314034,-1.333794,7.497038
9,Afghanistan,12,-1.431590,-1.998775,-0.687488,-1.360174,7.497755
18,Albania,10,-0.399795,0.442664,-0.517345,0.618363,9.302960
19,Albania,11,-0.669036,0.449130,-0.127154,0.657933,9.337532


# Machine Learning Overview

## Concepts

See the [main document](http://m-clark.github.io/data-processing-and-visualization/ml.html)

## Techniques

### Regularized Regression

In [12]:
happy_model_base = sm.OLS.from_formula(
  'happiness_score ~ democratic_quality + generosity + log_gdp_per_capita',
  data = happy_processed
).fit_regularized(L1_wt = .25)

happy_model_base.params

Intercept            -2.198226
democratic_quality    0.419285
generosity            0.137807
log_gdp_per_capita    0.238840
dtype: float64

### Random Forests

### Neural Nets

## Interpreting the Black Box

## Summary